<div align="center">
    <h1>Project - Time Series Multi-Horizon Forecasting using Transformers - DATA931  </h1>
    <h2>Ikram IDDOUCH - Sami SBAI </h2>
    
[![Python](https://img.shields.io/badge/python-blue.svg)](https://shields.io/)
[![VisualStudioCode](https://img.shields.io/badge/VisualStudioCode-green.svg)](https://shields.io/)
</div>

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import numpy as np
import yfinance as yf
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [3]:
df = pd.read_csv('Data/SP500_2000.csv')
df.describe()

,Price,Close,High,Low,Open,Volume
count,6524,6523,6523,6523,6523,6523
unique,6524,6415,6403,6418,6417,6356
top,Ticker,1097.280029296875,1263.8599853515625,1274.8599853515625,1139.9300537109375,4223740000
freq,1,3,3,4,3,12


In [ ]:
df.info()